### 1 stacking 的基本思想


stacking 就是将一系列模型（也称基模型）的输出结果作为新特征输入到其他模型，这种方法由于实现了模型的层叠，即第一层的模型输出作为第二层模型的输入，第二层模型的输出作为第三层模型的输入，依次类推，最后一层模型输出的结果作为最终结果。本文会以两层的 stacking 为例进行说明。



stacking 的思想也很好理解，这里以论文审稿为例，首先是三个审稿人分别对论文进行审稿，然后分别返回审稿意见给总编辑，总编辑会结合审稿人的意见给出最终的判断，即是否录用。



对应于stacking，这里的三个审稿人就是第一层的模型，其输出（审稿人意见）会作为第二层模型（总编辑）的输入，然后第二层模型会给出最终的结果。



stacking 的思想很好理解，但是在实现时需要注意不能有泄漏（leak）的情况，也就是说对于训练样本中的每一条数据，基模型输出其结果时并不能用这条数据来训练。



否则就是用这条数据来训练，同时用这条数据来测试，这样会造成最终预测时的过拟合现象，即经过stacking后在训练集上进行验证时效果很好，但是在测试集上效果很差。



为了解决这个泄漏的问题，需要通过 K-Fold 方法分别输出各部分样本的结果，这里以 5-Fold 为例，具体步骤如下



(1) 将数据划分为 5 部分，每次用其中 1 部分做验证集，其余 4 部分做训练集，则共可训练出 5 个模型

(2) 对于训练集，每次训练出一个模型时，通过该模型对没有用来训练的验证集进行预测，将预测结果作为验证集对应的样本的第二层输入，则依次遍历5次后，每个训练样本都可得到其输出结果作为第二层模型的输入

(3) 对于测试集，每次训练出一个模型时，都用这个模型对其进行预测，则最终测试集的每个样本都会有5个输出结果，对这些结果取平均作为该样本的第二层输入

除此之外，用 stacking 或者说 ensemble 这一类方法时还需要注意以下两点：

Base Model 之间的相关性要尽可能的小，从而能够互补模型间的优势

Base Model 之间的性能表现不能差距太大，太差的模型会拖后腿

### 2. common interface for K-Fold

In [35]:
import numpy as np
from sklearn.model_selection import KFold

In [36]:
# load train data and test data as ndarray
# a m×n ndarray means that there are m samples， while each sample has n dimension feature
x_train_file = './data/selectedFeatures/X_train_select.npy'
y_train_file = './data/selectedFeatures/label.npy'
x_test_file = './data/selectedFeatures/X_test_select.npy'
x_train = np.load(x_train_file).astype(np.float)
y_train = np.load(y_train_file).astype(np.float)
x_test = np.load(x_test_file).astype(np.float)
print(x_train.shape, y_train.shape, x_test.shape)

FileNotFoundError: [Errno 2] No such file or directory: './data/selectedFeatures/X_train_select.npy'

In [24]:
class BasicModel(object):
    """Parent class of basic models"""
    def train(self, x_train, y_train, x_val, y_val):
        """return a trained model and eval metric of validation data"""
        pass

    def predict(self, model, x_test):
        """return the predicted result"""
        pass
    
    def get_oof(self, x_train, y_train, x_test, n_folds = 5):
        """K-fold stacking"""
        num_train, num_test = x_train.shape[0], x_test.shape[0]
        oof_train = np.zeros((num_train,))
        oof_test = np.zeros((num_test,))
        oof_test_all_fold = np.zeros((num_test, n_folds))
        aucs = []
        KF = KFold(n_splits = n_folds, random_state=2017)
        for i, (train_index, val_index) in enumerate(KF.split(x_train)):
            print('{0} fold, train {1}, val {2}'.format(i, len(train_index), len(val_index)))
            x_tra, y_tra = x_train[train_index], y_train[train_index]
            x_val, y_val = x_train[val_index], y_train[val_index]
            model, auc = self.train(x_tra, y_tra, x_val, y_val)
            aucs.append(auc)
            oof_train[val_index] = self.predict(model, x_val)
            oof_test_all_fold[:, i] = self.predict(model, x_test)
        oof_test = np.mean(oof_test_all_fold, axis=1)
        print('all aucs {0}, average {1}'.format(aucs, np.mean(aucs)))
        return oof_train, oof_test

create two models for first-layer stacking: xgb and lgb

In [28]:
import xgboost as xgb
class XGBClassifier(BasicModel):
    def __init__(self):
        """set parameter"""
        self.num_rounds=1000
        self.early_stopping_round = 15
        self.params = {
            'objective': 'binary:logistic',
            'eta': 0.1,
            'max_depth': 8,
            'eval_metric': 'auc',
            'seed': 0,
            'silent': 0
        }
        
    def train(self, x_train, y_train, x_val, y_val):
        print('train with xgb model')
        xgbtrain = xgb.DMatrix(x_train, y_train)
        xgbval = xgb.DMatrix(x_val, y_val)
        watchlist = [(xgbtrain, 'train'), (xgbval, 'val')]
        model = xgb.train(self.params,
                          xgbtrain,
                          self.num_rounds,
                          watchlist,
                          early_stopping_rounds = self.early_stopping_rounds)
        return model, float(model.eval(xgbval).split()[1].split(':')[1])
    
    def predict(self, model, x_test):
        print('test with xgb model')
        xgbtest = xgb.DMatrix(x_test)
        return model.predict(xgbtest)

In [33]:
import lightgbm as lgb
class LGBClassifier(BasicModel):
    def __init__(self):
        self.num_boost_round = 2000
        self.early_stopping_round = 15
        self.params = {
            'task': 'train',
            'boosting_type': 'dart',
            'objective': 'binary',
            'metric': {'auc', 'binary_logloss'},
            'num_leaves': 80,
            'learning_rate': 0.05,
            'feature_fraction': 0.5,
            'bagging_fraction': 1,
            'bagging_freq': 5,
            'max_bin': 300,
            'is_unbalance': True,
            'lambda_l2': 50,
            'verbose': -1
        }
    
    def train(self, x_train, y_train, x_val, y_val):
        print('train with lgb model')
        lgbtrain = lgb.Dataset(x_train, y_train)
        lgbval = lgb.Dataset(x_val, y_val)
        model = lgb.train(self.params,
                         lgbtrain,
                         valid_sets=lgbval,
                         verbose_eval=self.num_boost_round,
                         num_boost_round=self.num_boost_round,
                         early_stopping_rounds=self.early_stopping_rounds)
        return model, model.best_score['valid_0']['auc']
    
    def predict(self, model, x_test):
        print('test with lgb model')
        return model.predict(x_test, num_iteration=model.best_iteration)

下一个步骤就是将这两个基模型的输出作为第二层模型的输入，这里选用的第二层模型是 LogisticsRegression，

首先需要将各个基模型的输出 reshape 和 concatenate 成合适的大小

In [34]:
# get output of first layer models and construct as input for the second layer
lgb_classifier = LGBClassifier()
lgb_oof_train, lgb_off_test = lgb_classifier.get_oof(x_train, y_train, x_test)
print(lgb_oof_train.shape, lgb_oof_test.shape)

xgb_classifier = XGBClassifier()
xgb_oof_train, xgb_oof_test = xgb_classifier.get_oof(x_train, y_train, x_test)
print(xgb_oof_train.shape, xgb_oof_test.shape)

input_train = [xgb_oof_train, lgb_oof_train]
input_test = [xgb_oof_test, lgb_oof_test]

stacked_train = np.concatenate([f.reshape(-1, 1) for f in input_train], axis=1)
stacked_test = np.concatenate([f.reshape(-1, 1) for f in input_test], axis=1)
print(stacked_train.shape, stacked_test.shape)

NameError: name 'x_train' is not defined

In [37]:
# use LR as the model of the second layer
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# split for validation
n = int(stacked_train.shape[0] * 0.8)
x_tra, y_tra = stacked_train[:n], y_train[:n]
x_val, y_val = stacked_train[n:], y_train[n:]
model = LinearRegression()
model.fit(x_tra,y_tra)
y_pred = model.predict(x_val)
print(metrics.roc_auc_score(y_val, y_pred))

# predict on test data
final_model = LinearRegression()
final_model.fit(stacked_train, y_train)
test_prediction = final_model.predict(stacked_test)

NameError: name 'stacked_train' is not defined